<h1 style="text-align: center; font-weight: bold;">Preprocessing</h1>

In order to make the data suitable for the model, we need to preprocess it. In this notebook, we will preprocess the data in the following steps:
- Merge the datasets
- Remove duplicates
- Handle missing values
- Handle outliers
- Encode categorical features

In [1]:
import pandas as pd
import numpy as np

<h2 style="font-weight: bold;">1. Load datasets and prepare data for merging</h2>

In [2]:
# metacritic
metacritic_df = pd.read_csv("./MetaCritic/MetaCritic.csv")
print(f"MetaCritic: {metacritic_df.shape}")
print(metacritic_df.columns)

# opencritic
opencritic_df = pd.read_csv("./OpenCritic/game_data.csv")
print(f"OpenCritic: {opencritic_df.shape}")
print(opencritic_df.columns)

# steam
steam_df = pd.read_csv("./steam_scraping/data/outputs/steam_data.csv")
print(f"Steam: {steam_df.shape}")
print(steam_df.columns)

MetaCritic: (24258, 8)
Index(['Game Title', 'Game Genre', 'Pricing', 'Developer', 'Release Date',
       'Platform', 'Rating', 'Number of Ratings'],
      dtype='object')
OpenCritic: (7771, 9)
Index(['Game Title', 'Game Genre', 'Pricing', 'Game Library Size', 'Publisher',
       'Release Date', 'Platform', 'Rating', 'Number of Rating'],
      dtype='object')
Steam: (58320, 8)
Index(['Game Title', 'Game Genre', 'Pricing', 'Publisher', 'Release Date',
       'Platform', 'Rating', 'Number of Ratings'],
      dtype='object')


There are a few columns in the datasets whose column names are not the same. We will rename these columns to make them consistent across the datasets. We will also remove the columns that are not useful for the model.

- **OpenCritic**
    - Has 'Number of Rating' instead of 'Number of Ratings' and 'Publisher' instead of 'Developer'.
    - Has an additional 'Game Library Size' column which is not needed.

- **Steam**
    - Has 'Publisher' instead of 'Developer'

- **MetaCritic** is fine as it is, so we will leave it there.

In [3]:
opencritic_df.rename(columns={"Number of Rating" : "Number of Ratings",
                              "Publisher" : "Developer"}, inplace=True)
opencritic_df.drop(columns=["Game Library Size"], inplace=True)

steam_df.rename(columns={"Publisher" : "Developer"}, inplace=True)

print(opencritic_df.columns)
print(steam_df.columns)
print(metacritic_df.columns)

Index(['Game Title', 'Game Genre', 'Pricing', 'Developer', 'Release Date',
       'Platform', 'Rating', 'Number of Ratings'],
      dtype='object')
Index(['Game Title', 'Game Genre', 'Pricing', 'Developer', 'Release Date',
       'Platform', 'Rating', 'Number of Ratings'],
      dtype='object')
Index(['Game Title', 'Game Genre', 'Pricing', 'Developer', 'Release Date',
       'Platform', 'Rating', 'Number of Ratings'],
      dtype='object')


<h3 style="font-weight: bold;">Normalize <i>Ratings</i> column</h2>

Since our dataset is merged from 3 different sources, the ratings scale may be different. We will normalize the ratings to a scale of 0-100.

In [4]:
def getMinMaxRatings(df):
    return df['Rating'].max(), df['Rating'].min()


meta_max, meta_min = getMinMaxRatings(metacritic_df)
open_max, open_min = getMinMaxRatings(opencritic_df)
steam_max, steam_min = getMinMaxRatings(steam_df)

print(f"MetaCritic: Max: {meta_max}, Min: {meta_min}")
print(f"OpenCritic: Max: {open_max}, Min: {open_min}")
print(f"Steam: Max: {steam_max}, Min: {steam_min}")

MetaCritic: Max: 10.0, Min: 0.2
OpenCritic: Max: 97.0, Min: 12.0
Steam: Max: 100.0, Min: 0.0


As seen from the output above, **MetaCritic** have ratings in the scale of 0-10, while the other 2 are in the scale of 0-100. We will have to normalize **MetaCritic**'s ratings to 0-100.

In [5]:
metacritic_df['Rating'] = metacritic_df['Rating'] * 10

meta_max, meta_min = getMinMaxRatings(metacritic_df)
print(f"MetaCritic: Max: {meta_max}, Min: {meta_min}")

MetaCritic: Max: 100.0, Min: 2.0


We should also check for validity of data (e.g. if the ratings are within the expected range).

But since we have seen the minimum and maximum values of the ratings in the previous notebook, we can skip this step.

<h2 style="font-weight: bold;">2. Merging data</h2>

We will merge the datasets by `Game Title` column, which is common in all datasets.

According to dataset sizes, the priority order will be:
1. Steam (58k rows)
2. OpenCritic (24k rows)
3. MetaCritic (7k rows)

Based on the priority order, we will merge the datasets from **OpenCritic** and **MetaCritic** into **Steam**'s.
If the values in **Steam** are missing, we will fill them with the values from **OpenCritic** and **MetaCritic**.

In [6]:
# Merge dataframes
merged_df = pd.merge(steam_df, metacritic_df,
    on="Game Title",
    how="outer",
    suffixes=('_steam', '_metacritic')
)
merged_df = pd.merge(merged_df,opencritic_df,
    on="Game Title",
    how="outer",
    suffixes=('','_opencritic')
)

target_columns = ['Game Title', 'Game Genre', 'Pricing', 'Developer', 'Release Date', 'Platform', 'Rating', 'Number of Ratings']

# Fill values based on priority
for col in target_columns[1:]:
    # strip whitespace and [, ], ' and " from the columns as well
    merged_df[col] = merged_df[f"{col}_steam"].fillna(merged_df[f"{col}_metacritic"], axis=0).fillna(merged_df[col], axis=0)
    
string_col = ['Game Title', 'Game Genre', 'Pricing', 'Developer', 'Platform', 'Release Date']
for col in string_col:
    merged_df[col] = merged_df[col].str.strip().str.strip("[").str.strip("]").str.strip("'").str.strip("'").str.replace('\u202A', '', regex=False)



merged_df = merged_df[target_columns]
merged_df

,Game Title,Game Genre,Pricing,Developer,Release Date,Platform,Rating,Number of Ratings
0,! That Bastard Is Trying To Steal Our Gold !,"Action, Adventure, Casual, Indie",$2.99,WTFOMGames,Mar 1 2016,NaN,56.0,66.0
1,! Wild Russia !,"Action, Adventure, Casual",$19.99,Andreev Worlds,Apr 28 2020,NaN,61.0,60.0
2,!4RC4N01D!,Arcade,NaN,armogames,"Jan 12, 2018",PC,40.0,4.0
3,!4RC4N01D! 2: Retro Edition,Arcade,NaN,armogames,"Feb 6, 2018",PC,38.0,4.0
4,!4RC4N01D! 2: Retro Edition,Arcade,NaN,armogames,"Feb 6, 2018",PC,38.0,4.0
...,...,...,...,...,...,...,...,...
79856,🔴 Circles,"Casual, Indie",Free To Play,Jeroen Wimmers,Feb 17 2017,NaN,89.0,226.0
79857,🔴 Circles,"Casual, Indie",Free To Play,Jeroen Wimmers,Feb 17 2017,NaN,89.0,226.0
79858,🔴 Circles,"Casual, Indie",Free To Play,Jeroen Wimmers,Feb 17 2017,NaN,89.0,226.0
79859,🚀 Human Rocket Person,"Action, Indie, Simulation",$1.99,2nd Studio,Nov 14 2018,NaN,95.0,43.0


In [7]:
merged_df.dtypes

Game Title            object
Game Genre            object
Pricing               object
Developer             object
Release Date          object
Platform              object
Rating               float64
Number of Ratings    float64
dtype: object

<h2 style="font-weight: bold;">3. Data Cleaning</h2>

<h3 style="font-weight: bold;">3.1. Remove duplicates</h2>

As seen from the above preview, the total records of the dataset is **79861** entries. However, there are duplicates in the datasets. We will have to remove them.

In [8]:
# strip the string
merged_df['Game Title'] = merged_df['Game Title'].str.strip()
# remove duplicates
merged_df = merged_df.drop_duplicates(subset=['Game Title'])
merged_df.shape

(62373, 8)

<h3 style="font-weight: bold;">3.2. Check for invalidity</h2>

Supossedly, the ratings should be within the range of 0-100. But since we have known the minimum and maximum values of the ratings do not violate this constraint in the previous section, we can skip this step.

All we need is to check whether the `Number of Ratings` column is valid, i.e. it should be > 0.

In [9]:
print(f"Min Rating: {merged_df['Rating'].min()}")

Min Rating: 0.0


There are indeed columns with `Number of Ratings` = 0. We will remove these rows.

In [10]:
merged_df = merged_df[merged_df['Rating'] > 0]
merged_df.shape

(61757, 8)

<h3 style="font-weight: bold;">3.3. Fill or remove missing values</h2>

Since we merged 3 datasets together, there will exist rows where values from all 3 datasets are missing, hence the **NaN** values. We will fill these missing rows with accordingly.

First, we will check the missing values in the dataset.

In [11]:
def getMissingValues(df: pd.DataFrame) -> pd.Series:
    return df.isnull().sum()

missing_values = getMissingValues(merged_df)
missing_values

Game Title               0
Game Genre            1638
Pricing              12545
Developer               11
Release Date             4
Platform             41866
Rating                   0
Number of Ratings        0
dtype: int64

**Developer, Release Date and Number of Ratings** have very few missing values (<20), we can safely remove them from the dataset without significant impact.

In [12]:
merged_df = merged_df.dropna(subset=['Developer', 'Release Date', 'Number of Ratings'])

**Game Genre** has *2238* missing values. We will fill these missing values with *mode* because it is a categorical feature.

In [13]:
# get all genre values
genres = merged_df['Game Genre'].str.split(',').explode().str.strip("[").str.strip("'").str.strip("'").str.strip("]").dropna().value_counts()

mode_genre = genres.idxmax()
merged_df.fillna({'Game Genre': mode_genre}, inplace=True)
print(f"Missing values in Game Genre replaced with mode value: {mode_genre}")

Missing values in Game Genre replaced with mode value:  Indie


**Pricing** column is skewed, so we will fill the missing values with the *median*.

But first, there are values in `Pricing` where they do not contain a numeric value (usually "Free to Play" for "Free", etc.), we will have to convert the column to a numeric type, with non-numeric (might be Free) values as 0.

In [14]:
import re

merged_df.columns
pricing_col = merged_df['Pricing']

numeric_pattern = re.compile(r'(?:\d+)+')
non_numeric_prices = pricing_col[~pricing_col.str.contains(numeric_pattern, na=False)].unique()

non_numeric_prices

array([nan, 'Free', 'Free To Play', '', 'Free to Play', 'Free Demo',
       'Free Mod', 'Play for Free!', 'Install Now', 'Play the Demo',
       'Play WARMACHINE: Tactics Demo'], dtype=object)

Now that we knew non-numeric values in the `Pricing` column are mostly *free* games, we will replace them with `0`.

We also need to convert the `Pricing` column to a numeric type. Preferably `float`.

In [15]:
# convert the non-numeric prices to 0
merged_df['Pricing'] = merged_df['Pricing'].replace(non_numeric_prices, '0')

# convert pricing to numeric
merged_df['Pricing'] = merged_df['Pricing'].str.replace(r'\$\s*', '', regex=True).astype(float)

There are still NaN values in the `Pricing` column. We will fill them with the median value since the column is skewed and does not follow a normal distribution.

In [16]:
median_pricing = merged_df['Pricing'].median()
merged_df.fillna({'Pricing' : median_pricing}, inplace=True)

**Platform** column has the most missing values, *41882*. We will fill these missing values with the *mode* since the column is categorical.

In [17]:
platforms = merged_df['Platform'].str.split(',').explode().str.strip().value_counts()
mode_platform = platforms.idxmax()
merged_df.fillna({'Platform': mode_platform}, inplace=True)
print(f"Missing values in Platform replaced with mode value: {mode_platform}")

Missing values in Platform replaced with mode value: PC


**Rating** column has *600* missing values. We will fill these missing values with *median* since the rating is skewed.

But first, we need to convert them all into numerical datatype.

In [18]:
merged_df['Rating'] = merged_df['Rating'].astype(float)
median_rating = merged_df['Rating'].median()
merged_df.fillna({'Rating': median_rating}, inplace=True)
print(f"Missing values in Rating replaced with median value: {median_rating}")

Missing values in Rating replaced with median value: 79.0


Now that we have filled/removed missing values, we will check again to make sure there are no missing values left.

In [19]:
missing_values = getMissingValues(merged_df)
missing_values

Game Title           0
Game Genre           0
Pricing              0
Developer            0
Release Date         0
Platform             0
Rating               0
Number of Ratings    0
dtype: int64

We will also check for the appropriate datatypes for each column.

In [20]:
merged_df.dtypes

Game Title            object
Game Genre            object
Pricing              float64
Developer             object
Release Date          object
Platform              object
Rating               float64
Number of Ratings    float64
dtype: object

**Number of Ratings** should be `uint64` instead of `float`.

In [21]:
merged_df['Number of Ratings'] = merged_df['Number of Ratings'].astype(np.uint64)

In [22]:
merged_df.dtypes

Game Title            object
Game Genre            object
Pricing              float64
Developer             object
Release Date          object
Platform              object
Rating               float64
Number of Ratings     uint64
dtype: object

<h3 style="font-weight: bold;">3.4. Handle outliers</h2>

`Number of Ratings` column is suspected to be highly skewed because there are some game titles with very few ratings while some received a lot of ratings. We will remove the outliers in this column.

Since we want to mitigate the effect of outliers, while retaining data, we will have to **winsorize (cap)** the data. The chosen approach is using Percentile method and capp the data at 99th percentile.

In [23]:
merged_df['Number of Ratings'].describe().round()

count      61742.0
mean        1782.0
std        40793.0
min            3.0
25%           17.0
50%           43.0
75%          186.0
max      8389102.0
Name: Number of Ratings, dtype: float64

In [24]:
lower_bound = merged_df['Number of Ratings'].quantile(0.05)
upper_bound = merged_df['Number of Ratings'].quantile(0.99)

merged_df['Number of Ratings'] = merged_df['Number of Ratings'].clip(lower=lower_bound, upper=upper_bound).astype(np.uint64)

merged_df['Number of Ratings'].describe().round()

count    61742.0
mean       769.0
std       3100.0
min          7.0
25%         17.0
50%         43.0
75%        186.0
max      24635.0
Name: Number of Ratings, dtype: float64

<h2 style="font-weight: bold;">4. Save cleaned data</h2>

Sort the data alphabetically by `Game Title` and save it as a CSV file.

In [25]:
merged_df = merged_df.sort_values(by="Game Title")
merged_df.to_csv("cleaned_data.csv", index=False, encoding='utf-8')
merged_df.head()

,Game Title,Game Genre,Pricing,Developer,Release Date,Platform,Rating,Number of Ratings
0,! That Bastard Is Trying To Steal Our Gold !,"Action, Adventure, Casual, Indie",2.99,WTFOMGames,Mar 1 2016,PC,56.0,66
1,! Wild Russia !,"Action, Adventure, Casual",19.99,Andreev Worlds,Apr 28 2020,PC,61.0,60
2,!4RC4N01D!,Arcade,0.00,armogames,"Jan 12, 2018",PC,40.0,7
3,!4RC4N01D! 2: Retro Edition,Arcade,0.00,armogames,"Feb 6, 2018",PC,38.0,7
5,!4RC4N01D! 3: Cold Space,Arcade,0.00,armogames,"Mar 8, 2018",PC,30.0,7
